# Hyper Parameter Tuning for predicting OnInvoice Discounts
Having identified that GBR model works well in previous section based on K fold cross validation scores and Training mean squared error , in this notebook we try to tune the model for better performance

We use bayesSearchCV to find the best parameters for our GBR model

In [1]:
#### IMPORTS

import pandas as pd
import numpy as np
from types import FunctionType
from sklearn.model_selection import train_test_split
import math
import matplotlib.pyplot as plt
# Importing sklearn methods
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from skopt import BayesSearchCV
import skopt
import pickle

In [2]:
# data2.xlsx is the data obtained after running the feature engineering code
# data3.csv : converted data2.xlsx to data3.csv because of easiness of use of csv files in webapps
path = r"C:\Users\NISARG\Desktop\mech\Finance\Maverick\CODE"   #change the path to your local path
df = pd.read_csv(path + "\data3.csv")


count = 0
index_test = list()
index_train = list()
for i in range(len(df['upper_limit'])):
    if(df['Discount_Total'][i]>df['upper_limit'][i]):
        count+=1
        index_test.append(i)
    else:
        index_train.append(i)
print(count/len(df['upper_limit']))

0.3232645073885446


In [3]:

'''
Here we get the train and test datasets based on the logic described above
'''

df_test = df.iloc[index_test]
df_test = df_test.reset_index()
df_train = df.iloc[index_train]
df_train = df_train.reset_index()     

In [4]:
'''
Encoding categorical variables here
'''


from sklearn.preprocessing import LabelEncoder
def encodeWithSave(highGTOData):
    lb_make = LabelEncoder()
    highGTOData['sdfc_Tier'] = lb_make.fit_transform(highGTOData['sdfc_Tier'])
    print(lb_make.classes_)
    np.save('classes_sdfc_tier.npy', lb_make.classes_)
    for i in range(len(highGTOData['GTO_2019'])):
        if(highGTOData['poc_image'][i]==0):
            highGTOData['poc_image'][i] = "Mainstream"
    highGTOData['poc_image'] = lb_make.fit_transform(highGTOData['poc_image'])
    np.save('classes_poc_image.npy', lb_make.classes_)
    highGTOData['segment'] = lb_make.fit_transform(highGTOData['segment'])
    np.save('classes_segment.npy', lb_make.classes_)
    highGTOData['sub_segment'] = lb_make.fit_transform(highGTOData['sub_segment'])
    np.save('classes_sub_segment.npy', lb_make.classes_)
    highGTOData['Product Set'] = lb_make.fit_transform(highGTOData['Product Set'])
    np.save('classes_Product Set.npy', lb_make.classes_)
    highGTOData['Brand'] = lb_make.fit_transform(highGTOData['Brand'])
    np.save('classes_Brand.npy', lb_make.classes_)
    highGTOData['Sub-Brand'] = lb_make.fit_transform(highGTOData['Sub-Brand'])
    np.save('classes_Sub_Brand.npy', lb_make.classes_)
    highGTOData['Pack_Type'] = lb_make.fit_transform(highGTOData['Pack_Type'])
    np.save('classes_Pack_Type.npy', lb_make.classes_)
    highGTOData['Returnalility'] = lb_make.fit_transform(highGTOData['Returnalility'])
    np.save('classes_Returnalility.npy', lb_make.classes_)
    highGTOData['province'] = lb_make.fit_transform(highGTOData['province'])
    np.save('classes_province.npy', lb_make.classes_)
    highGTOData['GTO_growth'] = highGTOData['Expected_GTO'] - highGTOData['GTO_2019']
    return highGTOData


def encode(highGTOData):
    lb_make = LabelEncoder()
    highGTOData['sdfc_Tier'] = lb_make.fit_transform(highGTOData['sdfc_Tier'])
    for i in range(len(highGTOData['GTO_2019'])):
        if(highGTOData['poc_image'][i]==0):
            highGTOData['poc_image'][i] = "Mainstream"
    highGTOData['poc_image'] = lb_make.fit_transform(highGTOData['poc_image'])
    highGTOData['segment'] = lb_make.fit_transform(highGTOData['segment'])
    highGTOData['sub_segment'] = lb_make.fit_transform(highGTOData['sub_segment'])
    highGTOData['Product Set'] = lb_make.fit_transform(highGTOData['Product Set'])
    highGTOData['Brand'] = lb_make.fit_transform(highGTOData['Brand'])
    highGTOData['Sub-Brand'] = lb_make.fit_transform(highGTOData['Sub-Brand'])
    highGTOData['Pack_Type'] = lb_make.fit_transform(highGTOData['Pack_Type'])
    highGTOData['Returnalility'] = lb_make.fit_transform(highGTOData['Returnalility'])
    highGTOData['province'] = lb_make.fit_transform(highGTOData['province'])
    highGTOData['GTO_growth'] = highGTOData['Expected_GTO'] - highGTOData['GTO_2019']
    return highGTOData


#df = encodeWithSave(df)



In [5]:
df_test = encode(df_test)
df_train = encode(df_train)

C:\Users\NISARG\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:



lowGTOData_train = df_train[df_train.GTO_2019<10000]
lowGTOData_test = df_test[df_test.GTO_2019<10000]

midGTOData_train = df_train[(df_train.GTO_2019>10000)&(df_train.GTO_2019<50000)]
midGTOData_test = df_test[(df_test.GTO_2019>10000)&(df_test.GTO_2019<50000)]

highGTOData_train = df_train[df_train.GTO_2019>50000]
highGTOData_test = df_test[df_test.GTO_2019>50000]

In [7]:
lowGTOData_train = lowGTOData_train.reset_index()
lowGTOData_test = lowGTOData_test.reset_index()
target_train = lowGTOData_train['OnInvoice Discount(LCU)']
target_test = lowGTOData_test['OnInvoice Discount(LCU)']
colsToKeep = ['Volume_2019' , 'Volume_2018'  , 'Expected_GTO'  , 'Expected_product_volume', 'profitability_indicator' , 'upper_limit'  ,'sdfc_Tier'  , 'loyalty_index' , 'Returnalility', 'market_cap' ]
features_train = lowGTOData_train[colsToKeep]
features_test = lowGTOData_test[colsToKeep]

# Parameters to focus on in Gradient Boosting Regressor

    1) loss :
        a. ls : least square regression
        b. lad : least absolute deviation
        c. huber : Combination of ls & lad
        d. quantile : Used in quantile regression
        
    2) learning_rate : Learning rate is used to shrink the contribution of each tree in GBR , we try values of learning rate from 0.05 to 0.3 with step of 0.05
    
    3) n_estimators : No of boostings performed . Larger the number , better is the fit , but it can lead to overfitting as well
    
    4) subsample : The fraction of samples to be used for fitting the individual base learners. If smaller than 1.0 this results in Stochastic Gradient Boosting. subsample interacts with the parameter n_estimators. Choosing subsample < 1.0 leads to a reduction of variance and an increase in bias.
    
    5) criterion : function to measure quality of split
        a. friedman_mse : mse with improvement by friedman
        b. mse : mean squared error
        
    6) max_depth : Maximum depth of the individual regression estimators. The maximum depth limits the number of nodes in the tree. The best value depends on the interaction of the input variables. We try max_depth from 2 to 8 with step of 1
    
    7) max_features : auto , sqrt , log2 
        a. auto : max_features = n_features
        b. sqrt : max_features = sqrt(n_features)
        c. log2 : max_features = log2(n_features) 

In [9]:
params = dict()
params['loss'] = ['ls', 'lad', 'huber', 'quantile']
params['learning_rate'] = [0.05,0.1,0.15,0.2,0.25,0.3]
params['n_estimators'] = [50,100,150,200,250,300,350,400]
params['subsample'] = [0.6,0.8,1]
params['criterion'] = ['friedman_mse', 'mse']
params['max_depth'] = [2,3,4,5,6,7,8]
params['max_features'] = ['auto', 'sqrt', 'log2']



In [10]:
kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=34234)
# define the search
search = BayesSearchCV(estimator=GradientBoostingRegressor(), search_spaces=params, n_jobs=-1, cv=kfold)

In [11]:
search.fit(features_train,target_train)
# report the best result
print(search.best_score_)
print(search.best_params_)

0.5805991782410778
OrderedDict([('criterion', 'mse'), ('learning_rate', 0.05), ('loss', 'huber'), ('max_depth', 7), ('max_features', 'auto'), ('n_estimators', 250), ('subsample', 0.8)])


# Best Parameters for low GTO Data
    1) criterion : mse
    
    2) learning_rate : 0.05
    
    3) loss : huber
    
    4) max_depth : 7
    
    5) max_features : auto
    
    6) n_estimators : 250
    
    7) subsample : 0.8

# Running ML Model with best parameters and saving it

In [8]:
import joblib


gbr = GradientBoostingRegressor(criterion = 'mse' , n_estimators=250, learning_rate=0.05, max_depth=7, max_features = 'auto', loss='huber' , subsample = 0.8)

gbr.fit(features_train,target_train)
kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=34234)
cross_val_scores = cross_val_score(gbr, features_train, target_train, cv=kfold )
mse_train = math.sqrt(mean_squared_error(target_train,gbr.predict(features_train)))
mse = math.sqrt(mean_squared_error(target_test, gbr.predict(features_test)))
print(cross_val_scores)
print(np.mean(cross_val_scores))
print(mse_train)
print(mse)


filename = 'lowGTOModel_OnInvoiceDiscount.sav'
joblib.dump(gbr, open(filename, 'wb'))




[0.55713378 0.58982931 0.60904525 0.53063144 0.60572675 0.55849369
 0.57606577 0.57743952 0.58919725 0.60498865]
0.5798551419673723
89.22967369868886
341.48362669898916


In [7]:
'''
Keeping the features as seen from EXPLORATORY DATA ANALYSIS
'''

midGTOData_train = midGTOData_train.reset_index()
midGTOData_test = midGTOData_test.reset_index()
target_train = midGTOData_train['Discount_Total']
target_test = midGTOData_test['Discount_Total']
colsToKeep = ['Volume_2019' , 'Volume_2018' ,'Volume_2019 Product' ,'Expected_GTO','Expected_product_volume' , 'profitability_indicator' , 'upper_limit'  ,'sdfc_Tier'  , 'loyalty_index' , 'Returnalility',  'inventory_lingering_factor', 'market_cap',
       'order_size']
features_train = midGTOData_train[colsToKeep]
features_test = midGTOData_test[colsToKeep]

In [15]:
kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=1)
# define the search
search = BayesSearchCV(estimator=GradientBoostingRegressor(), search_spaces=params, n_jobs=-1, cv=kfold)

In [16]:
search.fit(features_train,target_train)
# report the best result
print(search.best_score_)
print(search.best_params_)

0.4889008475019431
OrderedDict([('criterion', 'friedman_mse'), ('learning_rate', 0.05), ('loss', 'ls'), ('max_depth', 3), ('max_features', 'auto'), ('n_estimators', 300), ('subsample', 0.6)])


# Best Parameters for mid GTO Data
    1) criterion : friedman_mse
    
    2) learning_rate : 0.05
    
    3) loss : ls
    
    4) max_depth = 3
    
    5) max_features = auto
    
    6) n_estimators = 300
    
    7) subsample = 0.6

# Running model with best parameters and saving it

In [10]:
import joblib
gbr = GradientBoostingRegressor(criterion = 'friedman_mse' , n_estimators=300, learning_rate=0.05, max_depth=3, max_features = 'auto', loss='ls' , subsample = 0.6)
gbr.fit(features_train,target_train)
kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=34234)
cross_val_scores = cross_val_score(gbr, features_train, target_train, cv=kfold )
mse_train = math.sqrt(mean_squared_error(target_train,gbr.predict(features_train)))
mse = math.sqrt(mean_squared_error(target_test, gbr.predict(features_test)))
print(cross_val_scores)
print(np.mean(cross_val_scores))
print(mse_train)
print(mse)



filename = 'midGTOModel_OnInvoiceDiscount.sav'
joblib.dump(gbr, open(filename, 'wb'))

[0.62528884 0.30908704 0.49566408 0.28456212 0.40854459 0.69821543
 0.57775693 0.40176196 0.50746918 0.5111134 ]
0.48194635517864015
1333.8187381634866
6305.7931443586585


In [17]:
'''
Keeping the features as seen from EXPLORATORY DATA ANALYSIS
'''


highGTOData_train = highGTOData_train.reset_index()
highGTOData_test = highGTOData_test.reset_index()
target_train = highGTOData_train['Discount_Total']
target_test = highGTOData_test['Discount_Total']
colsToKeep = ['Volume_2019' , 'Volume_2018' ,'Volume_2019 Product' ,'Expected_GTO','Expected_product_volume' , 'profitability_indicator' , 'upper_limit'  ,  'inventory_lingering_factor',
       'order_size']
features_train = highGTOData_train[colsToKeep]
features_test = highGTOData_test[colsToKeep]

In [19]:
kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=34234)
# define the search
search = BayesSearchCV(estimator=GradientBoostingRegressor(), search_spaces=params, n_jobs=-1, cv=kfold)

In [20]:
search.fit(features_train,target_train)
# report the best result
print(search.best_score_)
print(search.best_params_)

C:\Users\NISARG\AppData\Local\Continuum\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


0.19788519762697399
OrderedDict([('criterion', 'friedman_mse'), ('learning_rate', 0.1), ('loss', 'ls'), ('max_depth', 5), ('max_features', 'sqrt'), ('n_estimators', 150), ('subsample', 0.8)])


# Best Parameters for High GTO Data
    1) criterion : friedman_mse
    
    2) learning_rate = 0.1
    
    3) loss : ls
    
    4) max_depth = 5
    
    5) max_features = sqrt
    
    6) n_estimators = 150
    
    7) subsample = 0.8

# Running model on best parameters and saving it

In [18]:
gbr = GradientBoostingRegressor(criterion = 'friedman_mse' , n_estimators=150, learning_rate=0.1, max_depth=5, max_features = 'sqrt', loss='ls' , subsample = 0.8)

gbr.fit(features_train,target_train)
kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=34234)
cross_val_scores = cross_val_score(gbr, features_train, target_train, cv=kfold )
mse_train = math.sqrt(mean_squared_error(target_train,gbr.predict(features_train)))
mse = math.sqrt(mean_squared_error(target_test, gbr.predict(features_test)))
print(cross_val_scores)
print(np.mean(cross_val_scores))
print(mse_train)
print(mse)


filename = 'HighGTOModel_OnInvoiceDiscount.sav'
joblib.dump(gbr, open(filename, 'wb'))

[-3.41858905  0.53594471  0.85195547  0.85052957  0.62786209 -0.72858824
  0.2885953  -1.38727719  0.78958495  0.94102403]
-0.06489583580942725
232.8254665083647
175100.22324594305
